# Définition du modèle Unet-3D

<p>Iddée : Nous allons implémenter une architecture 3D-UNET classique pour une image avec 2 channels d'entrée. Les images combinées_x inputs sont donc de dimension 128*128*128*2</p>

<li> Nous utilisons pour la compréhension du modèle le document suivant: U-Net and Its Variants for Medical Image Segmentation: A Review of Theory
and Applications</li>
<li>Ainsi qu'une référence du précédent document: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8768829&tag=1 </li>

## Schéma du modèle

<img src="Unet_scheme.png" width=60% height=50%  />

## Import des modules 
<li>Travaux de base de : @author: Sreenivas Bhattiprolu </li>


In [1]:
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.metrics import MeanIoU

## Travail sur l'initialisation 

<li>Initialisation des poids de chaques couches, conformément au cours de Stanford, nous utiliserons l'initialisation he_uniform qui s'explique par l'utilisation de fonction d'activation Relu</li>


In [2]:
################ Initialisation he uniforme qui est spécifique au fonction d'activation Relu#############
kernel_initializer = 'he_uniform' 
################## Autre possibilité Initialisation de Xavier commune au MOOC ######################
#kernel_initializer = 'glorot_uniform' 


## Définition de la structure du modèle

<li>Ici nous ne modifions pas le code source utilisé, cependant nous l'avons bien comparé au modèle théorique pour nous assurer de sa bonne structure</li>

In [3]:
#################### La Particularité de Keras et de TensorFlow est la définition static du modèle qui est réalisé ici (cf MOOC Stanford)################

def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH, IMG_CHANNELS, num_classes):

    #On définit la couche d'entré du modèle 
    # Keras: Création d'un tenseur d'entré avec les bonnes dimensions
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH, IMG_CHANNELS))
    s = inputs

    ##################On détaillera uniquement le premier bloc des deux phases ################

    #Phase de contraction ("détaillé en théorie dans papier rédigé")
    
    
    ################################### Bloc 1 #################################

    # Couche de convolution 3D: 16 nombre de Filtres de dimension 3*3*3, activation Relu, utilisation de l'initialisation précédente et on ajoute des zéro pour 
    # La taille des images d'origines. Note le Stride n'étant pas spécifier il est de 1. 
    # Pour voir les dimensions de la matrice sans les zéros padding nous pouvons faire le calcul suivant :
    # (N(taille input) - F(taille kernel)/ Stride) + 1 = (128 - 3)/1 + 1= 125 + 1 = 126
    c1 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
    # Couche de régularisation qui va éviter le surapprentissage en désactivement 10% des neuronnes
    c1 = Dropout(0.1)(c1)
    # Recommence une couche de convolution
    c1 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    # Le pooling 3D réduit la taille spatiale des caractéristiques tout en 
    # préservant les informations les plus importantes. Ici, nous utilisons 
    # un pooling de taille 2x2x2, ce qui signifie que nous prenons la valeur maximale dans 
    # chaque région de 2x2x2 pour réduire la taille des dimensions spatiales de moitié.
    p1 = MaxPooling3D((2, 2, 2))(c1)
    
    ################################### Bloc 2 #################################
    c2 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = MaxPooling3D((2, 2, 2))(c2)
     
    ################################### Bloc 3 #################################
    c3 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = MaxPooling3D((2, 2, 2))(c3)
    
    ################################### Bloc 4 #################################
    c4 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = MaxPooling3D(pool_size=(2, 2, 2))(c4)
    
    ################################### Bloc 5 #################################
    c5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)
    
    ################################### Phase d'expension #############################
    ############### La seule partie changeante  se trouve au niveau de la concaténation####### 
    ################################### Bloc 1 #################################
    u6 = Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4]) ########### La dimension est retrouvé en concaténant (annule la réduction  de dimension du maxPooling)##
    c6 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)
    ################################### Bloc 2 #################################
    u7 = Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)
    ################################### Bloc 3 #################################
    u8 = Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)
     ################################### Bloc 4 #################################
    u9 = Conv3DTranspose(16, (2, 2, 2), strides=(2, 2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)
    ################################### Bloc output #################################
    outputs = Conv3D(num_classes, (1, 1, 1), activation='softmax')(c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    #compile model outside of this function to make it flexible. 
    model.summary()
    
    return model




## Affichage du modèle 

In [4]:
#Test de l'affichage du modèle et de l'input/output
model = simple_unet_model(128, 128, 128, 2, 4)#Taille de nos images 128 128 128 2 numéro de classes 4 
print(model.input_shape)
print(model.output_shape)


2023-07-24 10:54:00.435942: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-07-24 10:54:00.435965: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-07-24 10:54:00.435971: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-07-24 10:54:00.436212: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-24 10:54:00.436238: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 128, 2)]   0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 128, 128, 128, 16)    880       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 128, 128, 128, 16)    0         ['conv3d[0][0]']              
                                                                                                  
 conv3d_1 (Conv3D)           (None, 128, 128, 128, 16)    6928      ['dropout[0][0]']             
                                                                                              